# This is an example of the Distributed Hash Table simulation
 - **author:** Isaque Lopes Campello


## Setup

In [1]:
import base64
import textwrap
import hashlib
from pathlib import Path
import numpy as np
import shutil
import os

from DHT_sim import *

## Demonstration of DHT

First, in order to have a DHT, we will create the nodes that will compose the network. For example, let's create a network with 5 nodes.

In [2]:
a = node("a")
b = node("b")
c = node("c")
d = node("d")
e = node("e")

Then, we need to create the network. To do this, we use the function *join_network(node)*, and pass in any node, and the node will be added to the correct place based on it's id, in order to create a circular network.

In [3]:
a.join_network(a)
b.join_network(a)
c.join_network(a)
d.join_network(a)
e.join_network(a)

In [4]:
for node in [a, b, c, d, e]:
    print(f"{node.name} id: {str(node.id)[:5]}..., neighbours: {node.neighbours[0].name}, {node.neighbours[1].name}, first: {node.first}")

a id: 91634..., neighbours: e, d, first: False
b id: 28106..., neighbours: c, e, first: False
c id: 21027..., neighbours: d, b, first: False
d id: 11159..., neighbours: a, c, first: True
e id: 28710..., neighbours: b, a, first: False


We also keep track of who is the *first* node, which is used in many operations. Bellow we can iterate over our nodes and move *clockwise*, always looking to our right neighbour, and see that we arrive back to the first node from which we began

In [5]:
node_list = ""
current_node = d
for i in range(6):
    node_list += f"{current_node.neighbours[0].name} -> "
    current_node = current_node.neighbours[0]
print(node_list)

a -> e -> b -> c -> d -> a -> 


Let's now seed a file into the network. This file is a short video in the **videos** folder. We will seed this video and split it into 5 parts, which will be distributed into the network. The node that seeds it doesn't really matter.

To do this, we pass both the **path** and the **file name** into the function, as well as the **number of parts** into which we want to split the video.

In [6]:
a.seed("videos/David_&_Goliath_animation.mp4", "David_&_Goliath_animation", 5)

Let's now check where the parts where stored

In [7]:
for folder in ["a", "b", "c", "d", "e"]:
    print(f"-> {folder}")
    for file in os.listdir(folder):
        filename = os.fsdecode(file)
        print(f"   | {filename}")

-> a
   | David_&_Goliath_animation_0.txt
   | David_&_Goliath_animation_1.txt
   | David_&_Goliath_animation_2.txt
-> b
-> c
   | David_&_Goliath_animation_3.txt
-> d
   | David_&_Goliath_animation_4.txt
-> e


What if node **d** wants to leave the network? It will attribute it's files to it's right neighbour

In [8]:
d.leave_network()

In [9]:
for node in [a, b, c, e]:
    print(f"{node.name} id: {str(node.id)[:5]}..., neighbours: {node.neighbours[0].name}, {node.neighbours[1].name}, first: {node.first}")


a id: 91634..., neighbours: e, c, first: False
b id: 28106..., neighbours: c, e, first: False
c id: 21027..., neighbours: a, b, first: True
e id: 28710..., neighbours: b, a, first: False


In [10]:
for folder in ["a", "b", "c", "e"]:
    print(f"-> {folder}")
    for file in os.listdir(folder):
        filename = os.fsdecode(file)
        print(f"   | {filename}")

-> a
   | David_&_Goliath_animation_0.txt
   | David_&_Goliath_animation_1.txt
   | David_&_Goliath_animation_2.txt
-> b
-> c
   | David_&_Goliath_animation_3.txt
   | David_&_Goliath_animation_4.txt
-> e


As we can see above, the files were re-assigned to node **c**. Should node **d** re-enter the network, it will regain responsabilities for it's original files.

Node **d** can also rejoin the network from any node and it will automatically join in the correct place. Let's add back node **d** from node **b**, for example

In [11]:
d.join_network(a)

In [12]:
d.neighbours[0].name
d.neighbours[1].name

'c'

In [13]:
for folder in ["a", "b", "c", "d", "e"]:
    print(f"-> {folder}")
    for file in os.listdir(folder):
        filename = os.fsdecode(file)
        print(f"   | {filename}")

-> a
   | David_&_Goliath_animation_0.txt
   | David_&_Goliath_animation_1.txt
   | David_&_Goliath_animation_2.txt
-> b
-> c
   | David_&_Goliath_animation_3.txt
-> d
   | David_&_Goliath_animation_4.txt
-> e


Finally, any node can fetch the entire file by calling the *leech* fucntion. This funcion takes the file name and the number of parts that the original file was split into. Lets have node **e** fetch the entire file

In [14]:
e.leech("David_&_Goliath_animation", 5)

In [15]:
for folder in ["a", "b", "c", "d", "e"]:
    print(f"-> {folder}")
    for file in os.listdir(folder):
        filename = os.fsdecode(file)
        print(f"   | {filename}")

-> a
   | David_&_Goliath_animation_0.txt
   | David_&_Goliath_animation_1.txt
   | David_&_Goliath_animation_2.txt
-> b
-> c
   | David_&_Goliath_animation_3.txt
-> d
   | David_&_Goliath_animation_4.txt
-> e
   | David_&_Goliath_animation.mp4


As we can see, node **e** now has the original file in it's folder. Should node **e** leave, this file won't be redistributed, as it's not part of the DHT.

In [16]:
e.leave_network()

In [17]:
for folder in ["a", "b", "c", "d"]:
    print(f"-> {folder}")
    for file in os.listdir(folder):
        filename = os.fsdecode(file)
        print(f"   | {filename}")

-> a
   | David_&_Goliath_animation_0.txt
   | David_&_Goliath_animation_1.txt
   | David_&_Goliath_animation_2.txt
-> b
-> c
   | David_&_Goliath_animation_3.txt
-> d
   | David_&_Goliath_animation_4.txt


Since node **e** left the network, it's folder was deleted, and the video file was not redistributed into the DHT

Finally, lets have all the files leave the network in order to close all the directories. The last node will already have left the network because it will have no other neighbours, but we will still call it just to delete the directory

In [18]:
a.leave_network()
b.leave_network()
c.leave_network()
d.leave_network()

Node already outside network
